## Common Generative AI Use Cases

In this lab, you will implement several common generative AI use cases.

In addition to being the most common types of generative AI applications, these techniques serve as the building blocks of many more complex applications.

## Objective

You will learn to use text prompting for the following tasks:

- Ideation
- Text Classification
- Text Summarization
- Text Extraction
- Question Answering


## Task 1. Initialize Vertex AI in a Colab Enterprise notebook


In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 113.4 MB/s eta 0:00:00


In [1]:
from inspect import cleandoc
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

In [2]:
PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-east4"
vertexai.init(project=PROJECT_ID, location=REGION)

## Task 2. Load a generative model


In [4]:
model = GenerativeModel("gemini-pro")

## Task 3. Ideation

Within the context of using a generative AI model, "ideation" means using the model to help you brainstorm new content: article titles, sections to include in a document, interview questions, etc.


In [5]:
# This is a case where you may want the model to be more creative or surprising in the content it generates.
# To enable that, create a GenerationConfig object by copying the code below to a new code cell and running it:
creative_gen_config = GenerationConfig(temperature=1, top_p=0.8)

prompt = "What are some strategies for overcoming writer's block?"

response = model.generate_content(prompt, generation_config=creative_gen_config)
print(response.text)

## Strategies for Overcoming Writer's Block:

**1. Break Down the Task:**

* **Divide your writing into smaller, more manageable chunks.** Instead of staring at a blank page and feeling overwhelmed, focus on completing one section at a time. This can help you make progress and build momentum.
* **Set realistic goals.** Don't try to write a perfect first draft. Aim for a rough draft that you can revise and improve later.

**2. Prime the Pump:**

* **Freewriting:** Write continuously for a set amount of time, without worrying about grammar, spelling, or making sense. This can help you get your creative juices flowing and overcome the initial hurdle of starting.
* **Brainstorming:** Generate ideas by listing everything that comes to mind about your topic. Don't censor yourself; just write down whatever ideas come to you.
* **Reading:** Immerse yourself in the work of other writers. Pay attention to their style, structure, and use of language. This can inspire you and give you new ideas.



In [6]:
prompt = "Provide ten interview questions for the role of prompt engineer."

response = model.generate_content(prompt, generation_config=creative_gen_config)
print(response.text)

## 10 Interview Questions for Prompt Engineering

**1. Describe your understanding of the role of a prompt engineer.**
**2. What are the key differences between a prompt engineer and a machine learning engineer?**
**3. What are some of the challenges you've faced while working as a prompt engineer? How did you overcome them?**
**4. Explain the iterative process of prompt engineering. How do you know when a prompt is successful?**
**5. Discuss the importance of understanding different NLP models and their capabilities when designing prompts.**
**6. How do you ensure your prompts are unbiased and do not perpetuate harmful stereotypes?**
**7. What are some of the tools and techniques you use to evaluate the effectiveness of your prompts?**
**8. Describe a project where you used prompt engineering to achieve a specific goal. What were the results?**
**9. How do you stay up-to-date with the latest advancements in the field of prompt engineering?**
**10. What are some of the ethical consider

## Task 4. Text Classification

In modern enterprises, a lot of work is dedicated to putting the right information in the right place. Routing customer support emails to the correct team or sending customer reviews to the relevant department are two examples. To do this, you could ask generative AI to classify those blocks of text to determine the right team.

Classification can have many possible applications, including:

- Sentiment analysis
- Topic classification
- Spam detection
- Intent recognition
- Language identification
- Toxicity detection
- Emotion detection


In [7]:
# For a more tightly-constrained summary, you may want to decrease the temperature and top_p parameters:
predictable_gen_config = GenerationConfig(temperature=0.1, top_p=0.1)

prompt = """
Given a user's input, classify their intent, such as "finding information", "making a reservation", or "placing an order". \n
user input: Hi, can you please book a table for two at Juan for May 1?
"""

response = model.generate_content(prompt, generation_config=predictable_gen_config)
print(response.text)

## Intent: Making a reservation

The user is requesting to book a table for two at a restaurant called Juan for May 1st. This falls under the category of "making a reservation". 



In [8]:
# You can help guide correct classifications of your data by providing examples:
prompt = """
What is the topic for a given news headline? \n
- business \n
- entertainment \n
- health \n
- sports \n
- technology \n\n

Text: Pixel 7 Pro Expert Hands On Review. \n
The answer is: technology \n

Text: Quit smoking? \n
The answer is: health \n

Text: Birdies or bogeys? Top 5 tips to hit under par \n
The answer is: sports \n

Text: Relief from local minimum-wage hike looking more remote \n
The answer is: business \n

Text: You won't guess who just arrived in Bari, Italy for the movie premiere. \n
The answer is:
"""

response = model.generate_content(prompt, generation_config=predictable_gen_config)
print(response.text)

entertainment 



In [9]:
# One great thing about defining a task as a classification task is that you can evaluate
# classification results very easily and meaningfully.

# Run this cell to import pandas as well as a classification_report metric from sklearn.
# You will also define some ground truth data for evaluation.

import pandas as pd
from sklearn.metrics import classification_report

review_data_df = pd.DataFrame(
    {
        "review": [
            "i love this product. it does have everything i am looking for!",
            "all i can say is that you will be happy after buying this product",
            "its way too expensive and not worth the price",
            "i am feeling okay. its neither good nor too bad.",
        ],
        "sentiment_groundtruth": ["positive", "positive", "negative", "neutral"],
    }
)

review_data_df

,review,sentiment_groundtruth
0,i love this product. it does have everything i...,positive
1,all i can say is that you will be happy after ...,positive
2,its way too expensive and not worth the price,negative
3,i am feeling okay. its neither good nor too bad.,neutral


In [10]:
# Now that you have the data with reviews and sentiments as ground truth labels,
# you can call the text generation model on each review row using the apply function.

# The get_sentiment function will be called to predict the sentiment for each row's review column,
# and the results will be stored in the sentiment_prediction column.


def get_sentiment(row):
    prompt = f"""Classify the sentiment of the following review as "positive", "neutral" or "negative". Return only the classification.
                review: {row}
              """
    response = model.generate_content(
        contents=prompt, generation_config=predictable_gen_config
    ).text
    return response


review_data_df["sentiment_prediction"] = review_data_df["review"].apply(get_sentiment)
review_data_df

,review,sentiment_groundtruth,sentiment_prediction
0,i love this product. it does have everything i...,positive,positive
1,all i can say is that you will be happy after ...,positive,positive
2,its way too expensive and not worth the price,negative,negative
3,i am feeling okay. its neither good nor too bad.,neutral,neutral


In [11]:
# Now you can call the classification_report function from sklearn to return classification metrics
# based on comparing the sentiment_groundtruth and predicted sentiment_prediction fields:

report = classification_report(
    review_data_df["sentiment_groundtruth"], review_data_df["sentiment_prediction"]
)
Markdown(report)

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00         1
     neutral       1.00      1.00      1.00         1
    positive       1.00      1.00      1.00         2

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4


## Task 5. Text Summarization

Sometimes you want to read the entirety of a text, and sometimes you just want the gist of it. Generative AI models can help create summaries of longer texts.


In [ ]:
# You may want to continue to use the more tightly-constrained temperature and top_p parameters we used above:
predictable_gen_config = GenerationConfig(temperature=0.1, top_p=0.1)

In [12]:
# Notice how this prompt's instructions to summarize also give some guidance on the length of the summary desired.
# Run the following code block in a new code cell to generate a short summary.

prompt = """
Provide a very short summary, no more than three sentences, for the following article:

Our quantum computers work by manipulating qubits in an orchestrated fashion that we call quantum algorithms.
The challenge is that qubits are so sensitive that even stray light can cause calculation errors — and the problem worsens as quantum computers grow.
This has significant consequences, since the best quantum algorithms that we know for running useful applications require the error rates of our qubits to be far lower than we have today.
To bridge this gap, we will need quantum error correction.
Quantum error correction protects information by encoding it across multiple physical qubits to form a "logical qubit," and is believed to be the only way to produce a large-scale quantum computer with error rates low enough for useful calculations.
Instead of computing on the individual qubits themselves, we will then compute on logical qubits. By encoding larger numbers of physical qubits on our quantum processor into one logical qubit, we hope to reduce the error rates to enable useful quantum algorithms.

Summary:

"""

response = model.generate_content(prompt, generation_config=predictable_gen_config)
print(response.text)

## Summary:

Quantum computers are sensitive to errors, which limits their usefulness. Quantum error correction aims to solve this by encoding information across multiple qubits, creating a more stable "logical qubit". This will allow for larger, more powerful quantum computers capable of running complex algorithms. 



In [13]:
# You might prefer your summary to be delivered as bullet points, displayed using Markdown to interpret the text formatting that Gemini returns:

prompt = """
Provide a very short summary in four bullet points for the following article:

Our quantum computers work by manipulating qubits in an orchestrated fashion that we call quantum algorithms.
The challenge is that qubits are so sensitive that even stray light can cause calculation errors — and the problem worsens as quantum computers grow.
This has significant consequences, since the best quantum algorithms that we know for running useful applications require the error rates of our qubits to be far lower than we have today.
To bridge this gap, we will need quantum error correction.
Quantum error correction protects information by encoding it across multiple physical qubits to form a "logical qubit," and is believed to be the only way to produce a large-scale quantum computer with error rates low enough for useful calculations.
Instead of computing on the individual qubits themselves, we will then compute on logical qubits. By encoding larger numbers of physical qubits on our quantum processor into one logical qubit, we hope to reduce the error rates to enable useful quantum algorithms.

Bullet points:

"""

response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
Markdown(response.text)

## Summary in four bullet points:

* **Quantum computers use qubits, which are sensitive to errors.** Even small disturbances like stray light can cause mistakes in calculations.
* **Current error rates are too high for useful applications.** The best quantum algorithms require much lower error rates than we have today.
* **Quantum error correction is the solution.** It encodes information across multiple qubits to create a "logical qubit" with lower error rates.
* **Logical qubits will enable useful quantum algorithms.** By using logical qubits, we can reduce errors and run complex calculations that are currently impossible. 


In [14]:
# Summarization can also be done on other forms of documents, like an email thread or the transcript of a conversation:

prompt = """
Please generate a summary of the following conversation and at the end summarize the to-do's for the support Agent:

Customer: Hi, I'm Larry, and I received the wrong item.

Support Agent: Hi, Larry. How would you like to see this resolved?

Customer: That's alright. I want to return the item and get a refund, please.

Support Agent: Of course. I can process the refund for you now. Can I have your order number, please?

Customer: It's [ORDER NUMBER].

Support Agent: Thank you. I've processed the refund, and you will receive your money back within 14 days.

Customer: Thank you very much.

Support Agent: You're welcome, Larry. Have a good day!

Summary:
"""

response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
Markdown(response.text)

The customer received the wrong item and wanted to return it for a refund. The support agent processed the refund and the customer will receive their money back within 14 days.

To-do's for the support agent:

* Process the refund for the customer.
* Send the customer an email confirmation of the refund.
* Close the ticket.

## Task 6. Text Extraction

Text Extraction is the process of pulling structured fields from unstructured text. By unstructured text, we mean text that lacks a computer-readable structure like CSV, JSON, or YAML, even if a human can identify some structure (like in the ingredient list of a recipe).

Some specific types of extraction include:

- **Named entity recognition (NER)**: Extract named entities from text, including people, places, organizations, and dates.
- **Relation extraction**: Extract the relationships between entities in text, such as family relationships between people.
- **Event extraction**: Extract events from text, such as project milestones and product launches.
- **Question answering**: Extract information from text to answer a question.


In [15]:
order = "We need eight grilled cheese sandwiches. Two with swiss cheese, three with muenster, three with cheddar."

prompt = """
    Break the customer's order into individual items with keys for the following fields:
    - item_name
    - cheese_selection

    Order:
    {order_field}
""".format(order_field=order)

response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
print(response.text)

```json
[
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Swiss"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Swiss"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Muenster"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Muenster"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Muenster"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Cheddar"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Cheddar"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Cheddar"
  }
]
```


**Note**: Gemini's [Function Calling](https://ai.google.dev/gemini-api/tutorials/extract_structured_data#use_function_calling) has capabilities to extract parameters according to their descriptions in order to allow you to call a function. While this is outside of the scope of this lab, you may want to investigate using these capabilities for Extraction, even if your end goal isn't to call a function.


## Task 7. Question Answering

Answering questions is one of the most common and most impressive uses of generative AI. The information the model returns could come from patterns in data the model was trained on or from additional information you provide the model as context.


In [16]:
# Run this code for an example of answering an Open Domain question, meaning its answer is
# publically available on the internet and does not require very recent or up-to-date knowledge.
# An answer that meets those qualifications may have been included in the model's training data:

prompt = """Q: Who was President of the United States in 1955?
              Which party did he belong to?
            A:
        """
response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
print(response.text)

Dwight D. Eisenhower was President of the United States in 1955. He belonged to the Republican Party.


In [17]:
# Run this code for an example of answering an closed domain question, meaning its answer may be private
# to your organization and therefore you need to provide that information to the model within the prompt's "context":

context = """
Storage and content policy \n
How durable is my data in Cloud Storage? \n
Cloud Storage is designed for 99.999999999% (11 9's) annual durability, which is appropriate for even primary storage and
business-critical applications. This high durability level is achieved through erasure coding that stores data pieces redundantly
across multiple devices located in multiple availability zones.
Objects written to Cloud Storage must be redundantly stored in at least two different availability zones before the
write is acknowledged as successful. Checksums are stored and regularly revalidated to proactively verify that the data
integrity of all data at rest as well as to detect corruption of data in transit. If required, corrections are automatically
made using redundant data. Customers can optionally enable object versioning to add protection against accidental deletion.
"""

question = "How is high availability achieved?"

prompt = f"""Answer the question given in the contex below:
Context: {context}?\n
Question: {question} \n
Answer:
"""

print("[Prompt]")
print(prompt)

print("[Response]")
response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
print(response.text)

[Prompt]
Answer the question given in the contex below:
Context: 
Storage and content policy 

How durable is my data in Cloud Storage? 

Cloud Storage is designed for 99.999999999% (11 9's) annual durability, which is appropriate for even primary storage and
business-critical applications. This high durability level is achieved through erasure coding that stores data pieces redundantly
across multiple devices located in multiple availability zones.
Objects written to Cloud Storage must be redundantly stored in at least two different availability zones before the
write is acknowledged as successful. Checksums are stored and regularly revalidated to proactively verify that the data
integrity of all data at rest as well as to detect corruption of data in transit. If required, corrections are automatically
made using redundant data. Customers can optionally enable object versioning to add protection against accidental deletion.
?

Question: How is high availability achieved? 

Answer:

[R

**Note**: Evaluating the correctness of question-answering systems can be challenging, but Vertex AI provides a [Generative AI evaluation service](https://cloud.google.com/vertex-ai/generative-ai/docs/models/determine-eval#question-answering) with some metrics tailor-made for this use case.


## Next Steps

- Study Gemini [Function Calling with Gemini](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling).
- Learn more about the [Generative AI evaluation service](https://cloud.google.com/vertex-ai/generative-ai/docs/models/determine-eval#question-answering).
